# Spark

---

Spark le permite distribuir datos y cálculos en clústeres con múltiples nodos (piense en cada nodo como una computadora separada). La división de sus datos facilita el trabajo con conjuntos de datos muy grandes porque cada nodo solo funciona con una pequeña cantidad de datos.

Como cada nodo trabaja en su propio subconjunto de los datos totales, también realiza una parte de los cálculos totales necesarios, de modo que tanto el procesamiento de datos como el cálculo se realizan en paralelo sobre los nodos del clúster. Es un hecho que la computación en paralelo puede hacer que ciertos tipos de tareas de programación sean mucho más rápidas.

El primer paso para usar Spark es conectarse a un clúster. Habrá una computadora, llamada _master_, que administrará la división de los datos y los cálculos. El _master_ está conectado al resto de las computadoras del clúster, que se denominan _worker_. El _master_ envía a los _workers_ datos y cálculos para que se ejecuten, y ellos envían sus resultados al _master_.

## Spark DataFrames

La estructura de datos central de Spark es el Resilient Distributed Dataset (RDD). Este es un objeto de bajo nivel que permite que Spark haga su magia al dividir los datos en varios nodos del clúster. Sin embargo, es difícil trabajar directamente con los RDD, por lo que usaremos Spark DataFrame construido sobre los RDD.

In [ ]:
import pyspark.sql.functions as F
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

### Spark Schema

Si conoce el tipo de dato es bueno definir el schema en Spark, para tener un mejor rendimiento. El schema se puede definir usando la clase `StructType()` que recibe una lista de los diferentes tipos de datos. Para cada variable(o columna) puedo definir el tipo de datos usando la clase `StructField("name", dataType, bool)` que recibe el nombre del campo, el tipo de dato y si los datos pueden ser nulos.

### Lazy evaluation

Spark no realiza ninguna transformación hasta que se solicita una acción. Básicamente a medida que agregamos más código, simplemente estamos actualizando el proceso que va a ejecutar.

### Crear columnas

Actualizar un Spark DataFrame es algo diferente a trabajar `Pandas` porque el Spark DataFrame es inmutable . Esto significa que no se puede cambiar y, por lo tanto, las columnas no se pueden actualizar en su lugar.

Por lo tanto, todos estos métodos devuelven un nuevo DataFrame.

En Spark puedes hacer esto usando el método `.withColumn()`, que toma dos parámetros. Primero, una cadena con el nombre de su nueva columna y, en segundo lugar, la nueva columna en sí.

La nueva columna debe ser un objeto de clase `Column`. Crear uno de estos es tan fácil como extraer una columna de su DataFrame usando `df.colName`.

En algunas ocasiones querrás cambiar algún valor o una variable basado en un condicional. Spark, proporciona algunas sentencias condicionales que actúan de forma similar a **_if_**, **_then_**, y **_else_**. Es posible realizar una declaración tradicional del estilo mencionado anteriormente, sin embargo, puede degradar el rendimiento.
Podemos usar entonces condicionales incorporados optimizados, tales como:
* `when(<if condition>, <then x>)`
* `otherwise()`

### Filtrar datos

Podemos usar el método `.filter()` para filtrar datos. Este método toma una expresión que seguirá la `WHERE` sentencia de una expresión SQL como un string, o una columna Spark con valores booleanos (`True`/`False`).

**Ejemplo**:

```

df.filter("column > 150").show()

df.filter(df.Column > 150).show()

```

### Seleccionar

La variante Spark para seleccionar es el método `.select()`. Este método toma varios argumentos, uno para cada columna que desee seleccionar. Estos argumentos pueden ser el nombre de la columna como una cadena(uno para cada columna) o un objeto de columna(usando `df.colName`). Cuando pasa un objeto de columna, puede realizar operaciones como sumar o restar en la columna para cambiar los datos que contiene, al igual que en el interior `.withColumn()`.

La diferencia entre los métodos `.select()` y `.withColumn()` es que `.select()` devuelve solo las columnas que especifique, mientras que `.withColumn()` devuelve todas las columnas del DataFrame además de la que definió. A menudo es una buena idea dejar columnas que no necesita al comienzo de una operación para no arrastrar datos adicionales mientras hace cálculos. En este caso, usaría `.select()` y no `.withColumn()`.

**Nota**:

* Cuando hacemos un cálculo o una operación dentro de un `.select()`, para nombrar el resultado podemos usar el método `.alias()`.
* Podemos usar un método `.selectExpr()` que tome expresiones SQL como un string.

### Funciones definidas por el usuario (UDF)

Es un método de Python que el usuario escribe para realizar un bit específico de lógica. Una vez escrito, el método se llama a través de `pyspark.sql.functions.udf`. El resultado se almacena como una variable y se puede llamar como una función de Spark normal.

**Ejemplo**:

```

def reverseString(mystr):
    return mystr[::-1]

udfReverseString = udf(reverseString, StringType())

```

La función `udf(func, dataType)` toma dos argumentos, la función que acaba de definir y el tipo de dato que Spark devolverá.

### Agrupación

Métodos de agrupación como `.min()`, `.max()`, `.count()`, entre otros, se conocen como `GroupedData`. Estos se pueden usar después de usar el método `.groupBy()`.

**Ejemplo**

```

df.groupBy("column").min("column").show()

```

Además de los métodos `GroupedData` también podemos usar un método `.agg()`. Este método permite pasar una expresión de columna agregada que usa cualquier de las funciones agregadas del submódulo `pyspark.sql.functions`. Este submódulo contiene muchas funciones, y toma el nombre de una columna.

### Join

Para unir DataFrames podemos usar el método `.join()`. Este método recibe tres argumentos, el primero es el DataFrame con el cual voy a unir, el segundo, `on` es el nombre de la(s) columna(s) por las cual voy a unir; y el tercero, el argumento `how` especifica el tipo de join a realizar.

Por otro lado, también puedo usar Spark SQL para unir dos DataFrames. Para eso registramos los DataFrames como tablas en Spark.

**Ejemplo**:

```
df1.createOrRemplaceTempView("df1")
df2.createOrRemplaceTempView("df2")

sql = """
      SELECT *
      FROM df1
      LEFT JOIN df2
      ON df1.column = df2.column
      AND df1.column1 = df2.column1
      """

df = spark.sql(sql)
```

### Eliminar columna(s)

Para eliminar columna(s) en Spark podemos usar el método `.drop()`. Depende de si es una columna o varias, podemos hacerlo de la siguiente manera:

```
# Una sola columna

df = df.drop("Column")

# Varias columnas

cols_to_drop = ["col1", "col2"]

df = df.drop(*cols_to_drop)

```

## Preprocesamiento

### Valores missing

En muchas ocasiones el set de datos tiene valores missing que tienen algún mecanismo en particular. Podemos usar varios métodos para detectar y tratar los valores missing:

* **.isNull()**: Devuelve `True` cuando la observación es un missing.
* **.fillna(value, subset=None)**: Para reemplazar datos missing por un valor definido. El primer parámetro es el valor a reemplazar, y el segundo, la lista de las columnas en donde se va a imputar el dato missing.
* **.dropna(how="any", thresh=None, subset=None)**: Para eliminar datos missing

### Variables categóricas

Algunas clases que nos ayudan a codificar datos categóricos en la etapa de preprocesamiento son:

* `StringIndexer(inputCol, outputCol)` Los valores de índice que se asignan es basado en la frecuencia relativa de forma descendente.
* `OneHotEncoder()`

### Creación de vector predictor (Assembling)

En Spark se debe consolidar las distantes variables independientes en una sola columna

* `VectorAssembler(inputCols, outputCol)`

## Entrenamiento